<div style="background-color:white;" >
<div style="clear: both; display: table;">
  <div style="float: left; width: 14%; padding: 5px; height:auto">
    <img src="img/TUBraunschweig_CO_200vH_300dpi.jpg" alt="TU_Braunschweig" style="width:100%">
  </div>
  <div style="float: left; width: 28%; padding: 5px; height:auto">
    <img src="img/TU_Clausthal_Logo.png" alt="TU_Clausthal" style="width:100%">
  </div>
  <div style="float: left; width: 25%; padding: 5px; height:auto">
    <img src="img/ostfalia.jpg" alt="Ostfalia" style="width:100%">
  </div>
  <div style="float: left; width: 21%; padding: 5px;">
    <img src="img/niedersachsen_rgb_whitebg.png" alt="Niedersachsen" style="width:100%">
  </div>
  <div style="float: left; width: 9%; padding: 5px;">
    <img src="img/internet_BMBF_gefoerdert_2017_en.jpg" alt="bmbf" style="width:100%">
  </div>
</div>
<div style="text-align:center">
<img src="img/ki4all.jpg" alt="KI4ALL-Logo" width="200"/>
</div>
</div>

# Synthetic Biomedical Data – Lesson 4: Non-Causal Variation
Part of the *Microcredit Artificial Data Generator* module.

➡️ [Back to Lesson 3c: Correlated Features](03c_correlated_features.ipynb)
➡️ [Module README](../README.md)

*Before continuing, please ensure you reviewed the prerequisites and learning goals in Lesson 1.*


# Lesson 4a: Understanding Non-Causal Variation and Confounding

### Recap
In previous lessons, you learned to generate synthetic biomedical datasets with:
- Informative features that carry true class signals
- Noise features that are irrelevant
- Correlated feature clusters mimicking biological pathways

These elements reflect **causal biology**: features that directly relate to the disease or outcome of interest. However, real-world biomedical data contains an additional layer of complexity that has nothing to do with biology: **non-causal variation**.

### Why this lesson matters: The hidden danger in biomedical data

Imagine a study where:
- Site A recruited mostly healthy controls
- Site B recruited mostly diseased patients
- Both sites used different measurement instruments

Your model achieves 95% accuracy! But is it learning biology or just detecting which site the sample came from? This is **confounding** – when a non-causal factor (site) is inadvertently associated with the outcome (disease status).

Non-causal variation is one of the most common reasons why:
- Models fail to generalize to new sites or cohorts
- Biomarker discoveries don't replicate in independent studies
- Published ML models never make it to clinical practice

This lesson teaches you to recognize, understand, and eventually simulate these artifacts so you can:
- Design robust evaluation strategies
- Test whether your method learns real biology or memorizes artifacts
- Develop models that generalize beyond the training cohort

### Key terms
- **Non-causal variation / artifacts**: systematic differences in data that are unrelated to the biological outcome of interest.
- **Batch effects**: technical variation from sample processing (e.g., different lab personnel, reagent lots, measurement dates).
- **Site effects**: systematic differences between data collection sites (e.g., hospitals, research centers).
- **Instrument effects**: variation from different measurement devices or protocols.
- **Recruitment artifacts**: imbalances in how patients were selected or enrolled (e.g., time periods, eligibility criteria).
- **Pseudo-class / spurious subgroup**: a group that looks like a biological class but is actually driven by non-causal factors.
- **Confounding**: when a non-causal factor correlates with (or even perfectly predicts) the outcome, creating a spurious association.
- **Perfect confounding**: when site/batch membership completely determines class label (e.g., all controls from Site A, all cases from Site B).
- **Partial confounding**: when site/batch membership is strongly but not perfectly associated with class.
- **Leakage (in this context)**: when a model learns to predict non-causal subgroup membership instead of the true biological outcome.
- **Distributional shift**: when test data comes from a different distribution than training data (e.g., new site, new instrument).

### What you'll learn
After completing this notebook, you will be able to:
- Distinguish between causal biological variation and non-causal artifacts.
- Identify common sources of non-causal variation in biomedical studies.
- Explain what confounding means and why it's dangerous for machine learning.
- Recognize the difference between perfect and partial confounding.
- Understand why standard random cross-validation fails under confounding.
- Articulate the consequences of models that learn artifacts instead of biology.
- Appreciate why synthetic data with controlled confounding is essential for method testing.

# Part 1: What is Non-Causal Variation?

## The Nature of Biomedical Data Collection

Biomedical data is rarely collected under perfectly controlled laboratory conditions. Instead, it accumulates through:
- **Multi-site studies**: Different hospitals or research centers contribute samples
- **Multi-batch processing**: Samples are processed in groups over time
- **Equipment upgrades**: Measurement technology changes during a study
- **Protocol variations**: Slight differences in sample handling or preparation
- **Time-dependent factors**: Seasonal effects, personnel changes, reagent lot variations

Each of these introduces **systematic differences** in the measured features that have nothing to do with the biological question.

## Causal vs. Non-Causal Features

To understand the distinction, consider a cancer classification problem:

### Causal variation (what we want to learn):
```
Gene X expression → Tumor growth → Cancer diagnosis
↑                                   ↑
Biological cause                    Outcome
```
- Gene X is **mechanistically related** to cancer
- Expression level **varies with** disease state
- Relationship holds across **different populations and settings**

### Non-causal variation (what we want to avoid learning):
```
Site A: Old instrument → Lower measured values
Site B: New instrument → Higher measured values

Site A recruited: 80% healthy, 20% cancer
Site B recruited: 20% healthy, 80% cancer
```
- Measurement differences are **technical artifacts**
- Site membership **correlates with** disease status by chance (or flawed study design)
- Relationship is **specific to this dataset** and won't generalize

## Visualization: The Confounding Trap

Let's visualize what happens when non-causal variation confounds the outcome:

```
Scenario 1: No Confounding (Ideal)
─────────────────────────────────
Site A:  🔵🔵🔵 🔴🔴🔴  (50% healthy, 50% cancer)
Site B:  🔵🔵🔵 🔴🔴🔴  (50% healthy, 50% cancer)
Site C:  🔵🔵🔵 🔴🔴🔴  (50% healthy, 50% cancer)

→ Site membership tells us nothing about disease status
→ Models must learn biology to classify correctly


Scenario 2: Perfect Confounding (Disaster)
──────────────────────────────────────────
Site A:  🔵🔵🔵🔵🔵🔵  (100% healthy)
Site B:  🔴🔴🔴🔴🔴🔴  (100% cancer)
Site C:  🔵🔵🔵🔵🔵🔵  (100% healthy)

→ Site membership perfectly predicts disease!
→ Model can achieve 100% accuracy by learning site markers
→ Biology is irrelevant


Scenario 3: Partial Confounding (Common in practice)
────────────────────────────────────────────────────
Site A:  🔵🔵🔵🔵🔵 🔴     (85% healthy, 15% cancer)
Site B:  🔵 🔴🔴🔴🔴🔴     (15% healthy, 85% cancer)
Site C:  🔵🔵 🔴🔴🔴🔴     (30% healthy, 70% cancer)

→ Site membership is informative but not deterministic
→ Model has incentive to learn both site markers AND biology
→ Hard to disentangle: which features are causal?
```

## Reflection Question 1

Consider a genomic study where:
- Site A collected samples in 2015 using instrument 1
- Site B collected samples in 2020 using a new instrument 2
- Both sites studied the same disease vs. healthy comparison

**Question**: List at least three sources of non-causal variation that could exist between these two sites, even if the underlying biology is identical.

<details>
<summary>Click to see example answers</summary>

**Technical sources**:
1. **Sequencing depth**: The different instruments could produce more/ fewer reads per sample
2. **Base calling accuracy**: Newer technology could have improved error rates
3. **GC bias patterns**: Different instruments have characteristic biases
4. **Library preparation protocols**: May have changed over 5 years

**Biological sources (non-causal to disease)**:
5. **Population demographics**: Different geographic regions may have genetic background differences
6. **Age distribution**: Enrollment criteria may have shifted
7. **Comorbidities**: Healthcare practice changes over time

**Operational sources**:
8. **Sample storage time**: Site A samples stored 5 years longer
9. **Personnel**: Different lab technicians, protocols, quality control
10. **Reagent lots**: Different batches of chemicals and enzymes

</details>

# Part 2: Common Sources of Non-Causal Variation

## 2.1 Batch Effects

**Definition**: Technical variation introduced by sample processing in groups (batches).

### Typical scenarios:
- **Microarray/RNA-seq**: Samples processed on different days or by different personnel
- **Mass spectrometry**: Instrument calibration drifts between runs
- **Flow cytometry**: Reagent lot changes, temperature variations
- **Clinical chemistry**: Different analyzer calibrations

### Characteristics:
- Affects **all features** in a batch simultaneously (though with varying strength)
- Can be **additive** (constant shift) or **multiplicative** (scaling)
- May have **non-linear effects** (batch × feature interactions)
- Often **partially confounded** with outcome if batches processed samples based on availability

### Example:
```
Batch 1 (January):  Controls processed first (40 samples)
Batch 2 (February): Mixed (20 controls, 20 cases)
Batch 3 (March):    Cases processed last (40 samples)

→ Batch membership is partially confounded with disease status
→ Model might learn: "High expression in batch 3 = cancer"
→ Reality: Batch 3 had different reagent lot that artificially inflated values
```

## 2.2 Site Effects

**Definition**: Systematic differences between data collection centers.

### Typical scenarios:
- **Multi-center clinical trials**: Different hospitals use different equipment
- **Consortium studies**: Each lab has its own protocols
- **Meta-analyses**: Combining published datasets from different sources

### Characteristics:
- Can include **all batch effect types** (technical + operational)
- May include **population differences** (demographics, genetics)
- Often have **strong confounding** if sites specialize (e.g., cancer center vs. general hospital)
- Hardest to correct because sites differ in **multiple unmeasured ways**

### Example:
```
Site A (University Hospital):    Recruited severe cases
Site B (Community Clinic):       Recruited mild cases + controls
Site C (Preventive Care Center): Recruited only healthy controls

→ Perfect confounding: Site C has 0% cases
→ Model learns to recognize Site C markers → predicts "healthy"
→ When applied to new Site D: model fails completely
```

## 2.3 Instrument and Protocol Effects

**Definition**: Variation from different measurement technologies or procedures.

### Typical scenarios:
- **Technology upgrades**: Study switches from microarray to RNA-seq mid-way
- **Vendor differences**: Roche vs. Abbott vs. Siemens diagnostic assays
- **Protocol versions**: Sample prep updated based on manufacturer recommendations

### Characteristics:
- Creates **discrete subgroups** in feature space
- May affect **specific feature subsets** (e.g., only low-abundance proteins)
- Confounding occurs if instrument change **coincides with** changes in patient recruitment

## 2.4 Time-Dependent Artifacts

**Definition**: Variation that correlates with time (recruitment year, processing date).

### Typical scenarios:
- **Seasonal effects**: Vitamin D levels vary by season
- **Enrollment period**: Early vs. late recruitment in longitudinal study
- **Secular trends**: Disease definition changes (e.g., diagnostic criteria updates)
- **Storage duration**: Sample degradation over years

### Why it's dangerous:
```
Year 1: Recruited 80% controls (study startup phase)
Year 2: Recruited 50% controls, 50% cases (full enrollment)
Year 3: Recruited 20% controls (mostly cases by end)

→ Year correlates with disease prevalence
→ Year also correlates with sample age (degradation)
→ Model learns temporal markers instead of disease biology
```

## Reflection Question 2

**Scenario**: You're analyzing a proteomics dataset where:
- Samples from 2018-2019 were measured on Instrument A
- Instrument A broke down in 2020
- Samples from 2020-2021 were measured on Instrument B (same technology, different unit)
- Disease prevalence was roughly constant across all years

**Question**: 
1. Is this an example of confounding? Why or why not?
2. Would random cross-validation give you an accurate estimate of model performance? Why?
3. What evaluation strategy would be more appropriate?

<details>
<summary>Click to see answer</summary>

**Answer 1**: **No, this is NOT confounding** (at least not with disease status). Since disease prevalence is constant across years, instrument membership does not predict the outcome. However, it is still a **distributional shift** that could affect generalization.

**Answer 2**: **Yes, random CV would likely give accurate estimates** because:
- Each CV fold would contain a mix of both instruments
- The model can learn instrument-invariant features
- Test set distribution matches training set distribution

**However**, random CV would NOT tell you whether the model generalizes to:
- A new Instrument C at a different site
- Samples measured exclusively on Instrument A or B

**Answer 3**: More appropriate strategies:
- **Instrument-aware CV**: Train on Instrument A, test on Instrument B (and vice versa)
- **Temporal CV**: Train on 2018-2019, test on 2020-2021
- **Sensitivity analysis**: Compare performance with and without instrument correction

This illustrates an important point: **Not all non-causal variation is confounding, but it can still pose generalization challenges.**

</details>

# Part 3: The Mechanics of Confounding

## What Makes Confounding Dangerous?

Confounding creates a **shortcut** for the model: instead of learning the hard task (biology), it learns the easy task (subgroup membership).

### The Shortcut Learning Problem

Consider two tasks:

**Task 1 (Hard)**: Learn biology
```
Input:  Gene expression (10,000 genes)
Goal:   Predict cancer vs. healthy
Signal: Subtle, distributed across multiple pathways
        Requires integration of information
        Noisy, overlapping distributions
```

**Task 2 (Easy)**: Learn site membership
```
Input:  Same gene expression data
Goal:   Predict site membership (A, B, or C)
Signal: Strong, consistent within-site patterns
        Technical artifacts affect ALL features simultaneously
        Clean separation in PCA/t-SNE
```

**If site membership predicts disease** → Model defaults to Task 2 because it's easier and gives better training performance!

## Perfect vs. Partial Confounding

### Perfect Confounding
```
Site A: 100% healthy (n=50)
Site B: 100% cancer  (n=50)
```

**Consequences**:
- Site membership **deterministically predicts** outcome
- Model can achieve **100% accuracy** without using any biology
- **Impossible to disentangle** site effects from disease effects
- Random CV gives **wildly optimistic** performance estimates

**Feature selection behavior**:
- Selectors pick features that differ between sites (technical artifacts)
- True biological features may be ignored if site effects are stronger

### Partial Confounding
```
Site A: 70% healthy, 30% cancer  (n=50)
Site B: 30% healthy, 70% cancer  (n=50)
```

**Consequences**:
- Site membership is **informative but not deterministic**
- Model learns a **mixture of site effects + biology**
- **Difficult to assess**: Which features are causal?
- Performance degrades gracefully on new sites (not catastrophic failure)

**Feature selection behavior**:
- Selectors pick a mix of technical artifacts and true biology
- Relative importance depends on effect sizes
- Instability increases: different CV folds may emphasize different features

## Mathematical Perspective

Let's formalize this:

**Observed data**: X (features), Y (disease status), S (site membership)

**Ideal scenario (no confounding)**:
```
P(Y|S) = P(Y)          Site doesn't predict disease
P(X|S) ≠ P(X)          But site affects measurements
Model learns: P(Y|X)   Uses biology, ignores site
```

**Confounded scenario**:
```
P(Y|S) ≠ P(Y)          Site predicts disease!
P(X|S) ≠ P(X)          Site affects measurements
Model learns: P(Y|S)   Uses site, ignores biology
```

**The model's perspective**:
- It seeks the path of least resistance (maximum likelihood)
- If P(Y|S) is easier to learn than P(Y|X_biology), it will learn site markers
- The model doesn't "know" which features are causal – it just optimizes the objective function

## Visualization: Decision Boundaries

```
Feature Space (2D simplified view)
──────────────────────────────────

Scenario A: No confounding
           Feature 2
              ↑
    Site A    |    Site B
    🔵  🔴    |    🔵  🔴     
    🔵  🔴    |    🔵  🔴
   ───────────|───────────→ Feature 1
    🔴  🔵    |    🔴  🔵
    🔴  🔵    |    🔴  🔵
              |

Decision boundary (biology): ─ ─ ─ ─ (horizontal)
Decision boundary (site):    │         (vertical)

→ Model must use Feature 2 (biology) to classify
→ Feature 1 (site marker) is uninformative


Scenario B: Perfect confounding
           Feature 2
              ↑
    Site A    |    Site B
    🔵  🔵    |    🔴  🔴     
    🔵  🔵    |    🔴  🔴
   ───────────|───────────→ Feature 1
    🔵  🔵    |    🔴  🔴
    🔵  🔵    |    🔴  🔴
              |

Decision boundary (biology): ─ ─ ─ ─ (could be anywhere)
Decision boundary (site):    │         (vertical)

→ Model learns site boundary (perfect separation!)
→ Biological signal is invisible/irrelevant
```

## Reflection Question 3

**Scenario**: You train a cancer classifier on a multi-site dataset with partial confounding:
- Your model achieves 85% AUC with random 5-fold CV
- When you apply it to a completely new site, performance drops to 55% AUC (barely better than random)

**Questions**:
1. What does this performance gap tell you about what your model learned?
2. Why did random CV fail to detect this problem?
3. If you had to estimate the "true" biological signal strength, would you trust the 85% or 55% estimate?

<details>
<summary>Click to see answer</summary>

**Answer 1**: The model learned **mostly site-specific artifacts**, not generalizable biology:
- The 85% AUC included ~30% from biology and ~55% from memorizing site patterns
- At a new site, site-specific features are completely unhelpful
- The remaining 55% AUC represents a mix of biology (signal) and lucky noise

**Answer 2**: Random CV failed because:
- Each training fold contained samples from **all sites**
- Each test fold also contained samples from **the same sites**
- The model could use site markers in training **and** test
- This mimics the training distribution but not real-world deployment (new sites)

**Answer 3**: **Neither estimate is trustworthy**:
- 85% is **too optimistic**: Inflated by site artifacts
- 55% is **too pessimistic**: Includes uncertainty from distributional shift (new site has different data distribution)

Better approach:
- Use **leave-one-site-out CV**: Train on sites A+B, test on site C (rotate)
- Average performance across held-out sites: e.g., 68% AUC
- This 68% better reflects generalizable biology
- The gap (85% - 68% = 17%) quantifies site-specific overfitting

</details>

# Part 4: Why Standard Cross-Validation Fails

## The Random CV Trap

Standard k-fold cross-validation randomly splits samples into training and test sets. This seems reasonable, but it **breaks down completely** under confounding.

### Example: Perfect Confounding

```
Dataset:
Site A: 50 healthy samples
Site B: 50 cancer samples
```

**Random 5-fold CV**:
```
Fold 1:
  Train: 40 from Site A (healthy) + 40 from Site B (cancer)
  Test:  10 from Site A (healthy) + 10 from Site B (cancer)
  
Model learns: "If feature X (site marker) is high → predict cancer"
Test performance: 100% accuracy!
```

**What went wrong?**
- Test set contains samples from **the same sites as training**
- Site markers work perfectly in both train and test
- CV reports 100% accuracy, suggesting perfect biology
- Reality: Model memorized technical artifacts

**Deployment to new Site C**:
```
Site C has different instruments, protocols, patient demographics
→ Site markers from A/B are irrelevant
→ Model performance: ~50% (random guessing)
```

## The Group CV Solution

**Principle**: Test and training sets must come from **different groups** (sites, batches).

### Leave-One-Site-Out CV (LOSO-CV)

```
Dataset:
Site A: 33 samples (17 healthy, 16 cancer)
Site B: 34 samples (16 healthy, 18 cancer)  
Site C: 33 samples (17 healthy, 16 cancer)
```

**LOSO-CV**:
```
Fold 1:
  Train: Sites A + B (67 samples)
  Test:  Site C (33 samples)

Fold 2:
  Train: Sites A + C (66 samples)
  Test:  Site B (34 samples)

Fold 3:
  Train: Sites B + C (67 samples)
  Test:  Site A (33 samples)
```

**Why it works**:
- Each test site is **completely unseen** during training
- Site-specific features are useless for prediction
- Only generalizable biology contributes to performance
- Mimics real-world deployment (new site/cohort)

**Trade-offs**:
- ✅ Honest estimate of generalization
- ✅ Forces model to learn robust features
- ❌ Requires multiple sites/batches (minimum 3)
- ❌ Smaller training sets per fold
- ❌ Higher variance in performance estimates

## Visual Comparison

```
Random CV (wrong for confounded data)
──────────────────────────────────────
Fold 1: [A A A B B | A B B C C]  ← Test has samples from all sites
Fold 2: [A A B B C | A A B C C]
Fold 3: [A B B C C | A A A B C]

→ Model can use site markers
→ Optimistic performance


Group CV (correct for confounded data)
───────────────────────────────────────
Fold 1: [B B B B C C C C | A A A A]  ← Test is entire site
Fold 2: [A A A A C C C C | B B B B]
Fold 3: [A A A A B B B B | C C C C]

→ Model cannot use site markers
→ Realistic performance
```

## Reflection Question 4

**Scenario**: You're designing a benchmark to test feature selection methods under batch effects.

You generate synthetic data:
- 100 samples (50 healthy, 50 disease)
- 10 informative features (true biology)
- 490 noise features
- 5 batches with strong batch effects but **balanced class distribution** in each batch

**Questions**:
1. Is this dataset confounded? Why or why not?
2. Would you expect random CV and batch-aware CV to give similar or different performance estimates?
3. Which CV strategy would better test feature selection stability?

<details>
<summary>Click to see answer</summary>

**Answer 1**: **No, this is not confounded** because class distribution is balanced across batches:
```
Batch 1: 10 healthy, 10 disease
Batch 2: 10 healthy, 10 disease
...
Batch membership does not predict outcome
```

**Answer 2**: **Similar performance**, but for different reasons:

*Random CV*:
- Model sees all batches in training and test
- Can learn to ignore batch effects or adjust for them
- Biology + batch correction → good performance

*Batch-aware CV*:
- Model trained on some batches, tested on others
- Must learn batch-invariant features (pure biology)
- Biology only → good performance (if strong signal)

**Answer 3**: **Batch-aware CV is better for stability testing**:
- Exposes whether selected features generalize across batches
- If a feature selector picks batch-specific noise features, batch-aware CV will show poor performance
- Random CV might hide this problem if the model can "memorize" batch patterns

**Key insight**: Even without confounding, non-causal variation poses a **generalization challenge**. Group CV tests the harder (and more realistic) generalization scenario.

</details>

# Part 5: Real-World Consequences

## Why This Matters for Biomedical Research

Non-causal variation and confounding aren't just theoretical concerns. They are responsible for many high-profile failures in biomedical machine learning.

### The Replication Crisis

**Published biomarker signatures that failed to replicate**:

1. **Cancer prognostic signatures** (early 2000s)
   - Multiple gene expression signatures published with 70-90% accuracy
   - Failed validation in independent cohorts (AUC dropped to 50-60%)
   - Post-hoc analysis: Signatures captured **batch effects** and **platform differences**

2. **COVID-19 diagnostic models** (2020-2021)
   - Hundreds of chest X-ray deep learning models published
   - Most learned to detect **image source** (hospital, scanner model) not COVID
   - Models worked perfectly on training hospital data, failed elsewhere

3. **Psychiatric disorder classification** (ongoing)
   - Brain imaging studies with 80-95% accuracy in single-site studies
   - Drop to 50-65% in multi-site validation
   - Site effects (scanner manufacturer, acquisition protocol) dominate biology

### Why Models Fail in Clinical Practice

Even when a model has "good" cross-validation performance, deployment fails because:

```
Training environment:
- Single hospital or consortium
- Consistent protocols
- Specific patient population
- Known equipment

→ Model learns: Biology + Local artifacts

Deployment environment:
- Different hospital
- Different protocols  
- Different population demographics
- Different equipment

→ Local artifacts are completely different
→ Model performance collapses
```

### Economic and Scientific Costs

**Failed clinical trials**:
- Millions spent validating biomarkers that don't generalize
- Patients enrolled in studies based on false predictions
- Opportunity cost: Better approaches not tested

**Wasted research effort**:
- Years spent developing models that memorize artifacts
- Follow-up studies chase spurious associations
- Incorrect biological hypotheses published and cited

**Loss of trust**:
- Clinicians skeptical of ML/AI after high-profile failures
- Funding agencies hesitant to support ML-heavy proposals
- Patients harmed by overconfident predictions

## The Pedagogical Value of Synthetic Confounding

**Why we need synthetic data with controlled confounding**:

### 1. Ground truth knowledge
- We **know** which features are causal (informative) vs. artifacts
- Can quantify: "Model used 30% biology, 70% batch markers"
- Impossible with real data where truth is unknown

### 2. Systematic testing
```
Vary confounding strength:
- No confounding (balanced design)
- Weak (60/40 class split per site)
- Moderate (70/30 split)
- Strong (90/10 split)  
- Perfect (100/0 split)

→ Observe model degradation curve
→ Identify which methods are robust
```

### 3. Safe learning environment
- Students make mistakes **before** working with real patient data
- Learn to spot warning signs of confounding
- Build intuition for evaluation strategies

### 4. Method comparison
- Benchmark feature selection methods under identical confounding
- Test correction strategies (ComBat, batch normalization)
- Validate CV strategies (random vs. group CV)

### 5. Reproducibility
- Deterministic seed → exactly reproducible experiments
- Others can verify claims about "confounding-robust method"
- Easier peer review and teaching

## Final Reflection Question

**Scenario**: You're reviewing a paper that claims a 10-gene signature predicts Alzheimer's disease with 92% accuracy.

The paper reports:
- Training data: 500 samples from 5 memory clinics
- Validation: 10-fold cross-validation
- Results: AUC = 0.92, sensitivity = 88%, specificity = 89%

**Questions**:
1. What critical information is missing from this description?
2. List at least three potential sources of confounding that could inflate these results.
3. What evaluation strategy would you recommend to test generalization?
4. If you could ask for one additional experiment, what would it be?

<details>
<summary>Click to see answer</summary>

**Answer 1 - Critical missing information**:
- **Class distribution per clinic**: Are some clinics mostly controls while others are mostly cases?
- **CV strategy**: Was it random 10-fold or leave-one-clinic-out?
- **Clinic characteristics**: Specialized memory clinics vs. general neurology?
- **When samples collected**: All at once or over years?
- **Measurement protocol**: Same for all clinics or heterogeneous?
- **Feature selection**: Done before CV (leakage) or within CV?
- **Population demographics**: Age, sex,... distribution per clinic?

**Answer 2 - Potential confounding sources**:
1. **Disease severity confounding**: 
   - Specialized memory clinics recruit late-stage Alzheimer's
   - General clinics recruit early-stage + controls
   - Model learns clinic markers → predicts disease stage, not disease presence

2. **Technical confounding**:
   - Different RNA extraction protocols per clinic
   - Different storage times (older samples degrade differently)
   - Different processing batches per clinic

3. **Population confounding**:
   - Clinic 1 in Scandinavia (high APOE ε4 frequency)
   - Clinic 2 in Asia (lower APOE ε4 frequency)
   - Geographic genetics correlate with both clinic and disease risk

4. **Recruitment bias**:
   - Early years: mostly controls (study establishment)
   - Later years: mostly cases (focused recruitment)
   - Sample age correlates with disease prevalence

**Answer 3 - Recommended evaluation**:
- **Leave-one-clinic-out CV**: 
  - Train on 4 clinics, test on 5th
  - Repeat for all 5 clinics
  - Report mean and range of AUC
- **Expected outcome**: If confounded, AUC drops to 0.65-0.75
- **If AUC remains >0.85**: Suggests robust biological signal

**Answer 4 - Additional experiment**:
**External validation on completely independent cohort**:
- New data from different country/healthcare system
- Different measurement platform (e.g., RNA-seq if original was microarray)
- Prospective collection (not retrospective)
- Pre-register analysis plan to prevent cherry-picking

**Red flags if authors refuse**:
- "External validation is too expensive"
- "Our signature only works with this specific protocol"
- "Other cohorts have different patient characteristics"

These are often signs the model learned dataset-specific artifacts, not biology.

</details>

# Summary

## Key Takeaways

### 1. Non-causal variation is ubiquitous in biomedical data
- **Batch effects**: Sample processing groups
- **Site effects**: Multi-center studies
- **Instrument effects**: Technology differences
- **Time effects**: Recruitment period, sample age

### 2. Confounding occurs when non-causal factors predict the outcome
- **Perfect confounding**: Complete overlap (e.g., Site A = 100% healthy)
- **Partial confounding**: Strong association (e.g., Site A = 80% healthy)
- **Distributional shift**: Different non-causal patterns in deployment

### 3. Models preferentially learn shortcuts over biology
- Technical artifacts often have **stronger signals** than biology
- Models optimize training objective, not causal understanding
- "Good" training performance can hide complete failure to learn biology

### 4. Random CV fails catastrophically under confounding
- Test set contains same sites/batches as training
- Gives **wildly optimistic** performance estimates
- Does not reflect real-world deployment scenarios

### 5. Group CV is essential for honest evaluation
- Leave-one-site-out, leave-one-batch-out
- Forces models to learn generalizable features
- Mimics true deployment (new site, new cohort)

### 6. Synthetic data enables systematic study
- Control confounding strength
- Know ground truth (causal vs. artifact features)
- Test methods under reproducible conditions
- Learn to recognize and avoid pitfalls

## What You've Learned

You should now be able to:
- Distinguish causal biological variation from technical artifacts
- Identify common sources of non-causal variation in study designs
- Explain why confounding is dangerous for ML models
- Recognize when random CV is inappropriate
- Articulate why group-aware CV is necessary
- Understand the value of synthetic confounded data for method testing

## Looking Ahead

In the next notebooks, you will:

**Lesson 4b: Generating Non-Causal Variation**
- Use `biomedical-data-generator` to add batch/site effects
- Create perfect and partial confounding scenarios
- Visualize confounding in PCA and t-SNE
- Control effect size of artifacts vs. biology

**Lesson 4c: Evaluation Pitfalls**
- Compare random CV vs. group CV empirically
- Measure performance degradation under confounding
- Implement proper nested CV with grouping
- Detect leakage from feature selection

**Lesson 4d: Method Testing**
- Benchmark feature selection stability under confounding
- Test batch correction methods (e.g., ComBat)
- Compare model robustness to distributional shift
- Design your own confounding robustness benchmark

## Next Steps

➡️ **Continue with: [Lesson 4b: Generating Non-Causal Variation](04b_non_causal_variation_generating.ipynb)**

In Lesson 4b, you will:
- Learn to add batch and site effects to synthetic data using `biomedical-data-generator`
- Create datasets with different confounding strengths
- Visualize how non-causal variation affects feature space
- Prepare data for evaluation experiments in Lesson 4c